# 台股財報擷取

網址：[公開資訊觀測站/財務報表/採IFRS後/合併/個別報表](https://mops.twse.com.tw/mops/web/t164sb03)

In [4]:
import requests, os, sqlite3, time
import pandas as pd

#
# 擷取特定日期財務報表
#
def crawl_data(sid,year,season,url):
    # 設定表單傳送值
    form_data = {
    'encodeURIComponent':1,
    'step':1,
    'firstin':1,
    'off':1,
    'queryName':'co_id',
    'inputType':'co_id',
    'TYPEK':'all',
    'isnew':'false',
    'co_id':sid,
    'year':year,
    'season':season,
    }
    
    try:
        data = requests.post(url, data=form_data)
    except Exception as e:
        print('**WARRN: 擷取失敗')
        print('錯誤訊息', e)
        return None
    
    # 如果資料為空，表示當天未交易，則返回
    if data.text == '':
        return None
    else:
        return data

    
#
# 處理擷取資料
#
def parse_data(data,sid,year,season,tablename):
    # 使用 pandas 讀取網頁裡的表格資料
    dfs = pd.read_html(data.text)

    # 取得資產負債表表格資料
    df = dfs[10]

    #重設欄位值
    df.columns = list(range(df.values.shape[1]))

    # 設定索引值
    df = df.set_index(0)

    # 只取第一欄資料(即當季財報資料), 及刪除 nan 項
    df = df[1].dropna()

    # 只取1個欄位資料後, df 變成 series, 需轉換為 Dataframe
    df = pd.DataFrame(df)

    # 轉置表格, 即列與欄交換
    df = df.transpose()

    # 刪除重覆欄位
    df = df.loc[:, ~df.columns.duplicated()]
    
    # 新增股票代碼及財報日期欄位
    df['sid'] = sid
    df['year'] = year
    df['season'] = season

    # 設定索引值
    df = df.set_index(['sid', 'year', 'season'])

    # 判斷 table 是否已存在
    query = "select count(*) from sqlite_master where type='table' and name='"+tablename+"'"
    exist = list(conn.execute(query))[0][0]

    # 如果 table 存在，新增資料，否則，建立表格及新增資料
    if exist:
        # 讀取資料庫中原有資料
        sqlstr = 'select * from ' + tablename
        data = pd.read_sql(sqlstr, conn, index_col=(['sid', 'year', 'season']))

        # 合併原有資料與新資料
        newdf = pd.concat([data, df], sort=True)

        # 刪除重覆項
        newdf.drop_duplicates(inplace=True)

        newdf.to_sql(tablename, conn, if_exists='replace')
        print(sid, year, '年第', season, '季',tablename,'新增成功')
    else:
        df.to_sql(tablename, conn, if_exists='replace')
        print(sid, year, '年第', season, '季',tablename,'儲存成功')

        
#------------------------------------
#輸入資產負債表擷取參數
print('擷取台股財報...')
sid = input('請輸入個股代號：')
syear = input('請輸入擷取起始年（請輸入民國年，譬如：108）：')
eyear = input('請輸入擷取結束年（請輸入民國年，譬如：108）：')

# 設定網址及 post 表單參數
bs_url = 'https://mops.twse.com.tw/mops/web/t164sb03'
income_url = 'https://mops.twse.com.tw/mops/web/t164sb04'
cash_url = 'https://mops.twse.com.tw/mops/web/t164sb05'

# 建立資料庫連線
conn = sqlite3.connect(os.path.join('data', 'stock1.db'))

for year in range(int(syear), int(eyear)+1):
    for i in range(1,5):
        season = '0' + str(i)

        # 資產負債表
        data = crawl_data(sid,str(year),season,bs_url)

        if data is None:
            print('資產負債表擷取資料失敗或無資料！')
        else:    
            parse_data(data,sid,str(year),season,'bs')

        # 綜合損益表
        data = crawl_data(sid,str(year),season,income_url)

        if data is None:
            print('綜合損益表擷取資料失敗或無資料！')
        else:    
            parse_data(data,sid,str(year),season,'income')

        # 現金流量表
        data = crawl_data(sid,str(year),season,cash_url)

        if data is None:
            print('現金流量表擷取資料失敗或無資料！')
        else:    
            parse_data(data,sid,str(year),season,'cashflow')

        time.sleep(1)
    
print('財報資料擷取完畢！')    

# 關閉資料庫連線
conn.close()

擷取台股財報...
請輸入個股代號：3056
請輸入擷取起始年（請輸入民國年，譬如：108）：104
請輸入擷取結束年（請輸入民國年，譬如：108）：108
3056 104 年第 01 季 bs 新增成功
3056 104 年第 01 季 income 新增成功
3056 104 年第 01 季 cashflow 新增成功
3056 104 年第 02 季 bs 新增成功
3056 104 年第 02 季 income 新增成功
3056 104 年第 02 季 cashflow 新增成功
3056 104 年第 03 季 bs 新增成功
3056 104 年第 03 季 income 新增成功
3056 104 年第 03 季 cashflow 新增成功
3056 104 年第 04 季 bs 新增成功
3056 104 年第 04 季 income 新增成功
3056 104 年第 04 季 cashflow 新增成功
3056 105 年第 01 季 bs 新增成功
3056 105 年第 01 季 income 新增成功
3056 105 年第 01 季 cashflow 新增成功
3056 105 年第 02 季 bs 新增成功
3056 105 年第 02 季 income 新增成功
3056 105 年第 02 季 cashflow 新增成功
3056 105 年第 03 季 bs 新增成功
3056 105 年第 03 季 income 新增成功
3056 105 年第 03 季 cashflow 新增成功
3056 105 年第 04 季 bs 新增成功
3056 105 年第 04 季 income 新增成功
3056 105 年第 04 季 cashflow 新增成功
3056 106 年第 01 季 bs 新增成功
3056 106 年第 01 季 income 新增成功
3056 106 年第 01 季 cashflow 新增成功
3056 106 年第 02 季 bs 新增成功
3056 106 年第 02 季 income 新增成功
3056 106 年第 02 季 cashflow 新增成功
3056 106 年第 03 季 bs 新增成功
3056 106 年第 03 季 income 新增成功
3056 106 年第 03 季 